In [ ]:
# -*- coding: utf-8 -*-

"""Contains function to perform INIT"""

from __future__ import absolute_import

import six

def init(model, epsilon, expression_profile=None):
    """
    Integrative Network Inference for Tissues (INIT)[1]
    
    Parameters
    ----------
    model: cobra.Model
        The constraint-based model to perform INIT on.
    epsilon: float
        A positive tolerance.
    expression_profile: ExpressionProfile
        The expression profile of the tissue.
        
    Returns
    -------
    
    References
    ----------
    .. [1]  Agren R, Bordel S, Mardinoglu A, Pornputtapong N, Nookaew I, et al. (2012)
            Reconstruction of Genome-Scale Active Metabolic Networks for 69 Human Cell
            Types and 16 Cancer Types Using INIT.
            PLoS Comput Biol 8(5): e1002518.
            doi:10.1371/journal.pcbi.1002518
    """
    
    assert isinstance(model, cobra.Model)
    assert isinstance(epsilon, float)
    assert isinstance(expression_profile, ExpressionProfile)
    
    obj_vars = []
    consts = []
    
    condition = expression_profile.conditions[0] if condition is None else condition
    not_measured_value = 0 if not_measured_value is None else not_measured_value
    reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)
    
    with model:
        prob = model.problem
        # Reactions
        for rxn_id, rxn_exp in six.iteritems(reaction_profile):
            rxn = model.reactions.get_by_id(rxn_id)
            y = prob.Variable("y_%s" % rxn_id, type="binary")
            const = prob.Constraint(rxn.flux_expression - (1000 * y),
                                    ub=0.0, name="rxn_const_%s" % rxn_id)
            epsilon_const = prob.Constraint(rxn.flux_expression + (1000 * (1 - y)) - epsilon,
                                            lb=0.0, name="rxn_epsilon_const_%s" % rxn_id)
            if rxn.reversibility is True:
                reversible_rxn_const = prob.Constraint(rxn.flux_expression,
                                                       lb=0.0, name="rev_rxn_const_%s" %rxn_id)
                consts.extend([y, const, epsilon_const, reversible_rxn_const])
            else:
                consts.extend([y, const, epsilon_const])
            # Metabolites
            for met in rxn.metabolites:
                x = prob.Variable("x_%s_%s".format(rxn_id, met.id), type="binary")
                met_const = prob.Constraint(rxn.metabolites.get_coefficient(met) - (1000 * x),
                                              ub=0.0, name="met_const_%s_%s".format(rxn_id, met.id))
                met_const_epsilon = prob.Constraint(rxn.metabolites.get_coefficient(met) + (1000 * (1 - x)) - epsilon,
                                                    lb=0.0, name="met_epsilon_const_%s_%s".format(rxn_id, met.id))
                met_coefficient_const = prob.Constraint(rxn.metbolites.get_coefficient(met),
                                                        lb=0.0, name="met_coefficient_const_%s_%s".format(rxn_id, met.id))
                consts.extend([met_const, met_const_epsilon, met_coefficient_const])
        
        model.add_cons_vars(consts)
        model.objective = prob.Objective(Zero, sloppy=True, direction="max")
        
        return model

In [1]:
import cobra

In [2]:
import cobra.test

In [3]:
model = cobra.test.create_test_model("ecoli")

In [9]:
for i in model.reactions.ABUTtex.metabolites:
    print()

{<Metabolite 4abut_e at 0x11a618198>: -1.0,
 <Metabolite 4abut_p at 0x11a6652e8>: 1.0}